In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install ultralytics 



In [ ]:
import os
from pathlib import Path
from IPython.display import Image, display
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO
print("✓ Packages installed successfully!")

In [ ]:
!ls /kaggle/input/

In [ ]:
DATASET_NAME = 'hand-detection'  
# Define paths
dataset_zip_path = f'/kaggle/input/hand-detection'
extract_path = '/kaggle/working/asl_dataset'

# Check if zip file exists
if os.path.exists(dataset_zip_path):
    print(f"✓ Found dataset at: {dataset_zip_path}")
else:
    print(f"✗ Dataset not found at: {dataset_zip_path}")
    print("\nPlease check:")
    print("1. You've uploaded the dataset to Kaggle")
    print("2. You've added it to this notebook using 'Add Data'")
    print("3. The DATASET_NAME variable matches your dataset folder name")

In [ ]:
dataset_path = f'/kaggle/input/{DATASET_NAME}'

print("Checking dataset structure...")
!ls {dataset_path}

print("\nDataset is ready to use!")

In [ ]:
# Fix data.yaml with absolute paths
import yaml
import os

# Read the original data.yaml
yaml_path = os.path.join(dataset_path, 'data.yaml')
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# Update paths to absolute paths
data['train'] = os.path.join(dataset_path, 'train/images')
data['val'] = os.path.join(dataset_path, 'valid/images')
data['test'] = os.path.join(dataset_path, 'test/images')

# Save the corrected data.yaml to working directory
output_yaml = '/kaggle/working/data.yaml'
with open(output_yaml, 'w') as f:
    yaml.dump(data, f, default_flow_style=False)

print("="*70)
print("✓ Fixed data.yaml created!")
print("="*70)
print("\nNew data.yaml content:")
print("-"*70)
with open(output_yaml, 'r') as f:
    print(f.read())
print("-"*70)
print(f"\n✓ Saved to: {output_yaml}")
print("\nDataset Info:")
print(f"  Classes: {data['nc']}")
print(f"  Names: {data['names']}")
print(f"  Train images: {data['train']}")
print(f"  Valid images: {data['val']}")
print(f"  Test images: {data['test']}")
print("="*70)


In [ ]:
import random
import cv2
from matplotlib import pyplot as plt

# Get sample images from training set
train_images_path = os.path.join(dataset_path, 'train', 'images')
sample_images = random.sample(os.listdir(train_images_path), 6)

# Display samples
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Sample Training Images', fontsize=16)

for idx, img_name in enumerate(sample_images):
    img_path = os.path.join(train_images_path, img_name)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    ax = axes[idx // 3, idx % 3]
    ax.imshow(img)
    ax.set_title(f"Image: {img_name[:10]}...")
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
MODEL_SIZE = 'yolo11n.pt'  # Change to yolo11s.pt, yolo11m.pt for better accuracy
EPOCHS = 100                # Increase to 100+ for better results
IMG_SIZE = 640             # Image size for training
BATCH_SIZE = 16            # Reduce to 8 if you get memory errors

print(f"Training Configuration:")
print("="*50)
print(f"Model: {MODEL_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Image Size: {IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Dataset: {yaml_path}")

In [ ]:
from ultralytics import YOLO

# Initialize YOLO model
model = YOLO(MODEL_SIZE)

print("✓ Model loaded successfully!")
print(f"Starting training for {EPOCHS} epochs...")
print("="*70)

In [ ]:
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    name='hands_yolo11',
    patience=10,           
    save=True,            
    device=0,              
    workers=2,             
    verbose=True,          
    plots=True          
)

print("\n" + "="*70)
print("✓ Training completed successfully!")
print("="*70)

In [ ]:
# Best model (use this one!)
best_model = '/kaggle/working/runs/detect/hands_yolo11/weights/best.pt'

# Last checkpoint
last_model = '/kaggle/working/runs/detect/hands_yolo11/weights/last.pt'

In [ ]:
import shutil
import os

# Create a clean folder in outputs
os.makedirs('/kaggle/working/saved_models', exist_ok=True)

# Copy your best model
shutil.copy(
    '/kaggle/working/runs/detect/hands_yolo11/weights/best.pt',
    '/kaggle/working/saved_models/hands_best.pt'
)

print("✓ Model saved to: /kaggle/working/saved_models/hands_best.pt")
print("✓ Will appear in Output tab after session ends")